In [1]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering

In [248]:
def read_dataframe_from_file(filename, filesubstr):
    
    final_results = []
    initial_line = 2 - 1
    initial_line_result = 5 - 1
    
    num_class_f = open('Final_Results/' + filesubstr + '/num_class.txt')
    for line in num_class_f:
        num_class = line
        
    #print(num_class)

    f = open(filename, 'r')
    tmp_arr = []
    for i, line in enumerate(f):
        if i == initial_line:
            line = line.replace('MoJo', '')
            line = line.replace('.rsf', '')
            line = line.replace('(', '')
            line = line.replace(')', '')
            line = line.split('_')
            name = line[0].split('/')[0]
            version = line[0].split('-')[-1]
            n_cluster = line[1]
            affinity = line[2]
            linkage = line[3]
            #print(line)
            #print('Version', version)
            initial_line += 5
            tmp_arr.append(name)
            tmp_arr.append(version)
            tmp_arr.append(n_cluster)
            tmp_arr.append(affinity)
            tmp_arr.append(linkage)
        elif i == initial_line_result:
            n_MoJo = line.split(' ')[-1].split('\n')[0]
            
            #print(line)
            initial_line_result += 5
            tmp_arr.append(n_MoJo)
            tmp_arr.append((1 - (float(n_MoJo) / float(num_class))) * 100)
            final_results.append(tmp_arr)
            tmp_arr = []
            
    
    
    f.close()
    return final_results

In [249]:
# Getting the smallest MoJo Value
rootdir = 'Final_Results/'
project_names = []

f = open('project_list.txt', 'r')
for line in f:
    print(line)
    project_names.append(line.split('\n')[0])

#project_names = ['apache-maven', 'apache-storm']
dir_arr = []

#Main clustering arr to be transformed to pandas dataframe later
cluster_main_arr = []

#Main CK arr to be transformed to pandas dataframe later
ck_sum_arr = []
ck_max_arr = []
ck_std_arr = []
ck_mean_arr = []

# Only considering the below ck_metrics
ck_metrics = ['cbo', 'wmc', 'dit', 'rfc', 'lcom',
       'totalMethods', 'staticMethods', 'publicMethods', 'privateMethods',
       'protectedMethods', 'defaultMethods', 'abstractMethods', 'finalMethods',
       'synchronizedMethods', 'totalFields', 'staticFields', 'publicFields',
       'privateFields', 'protectedFields', 'defaultFields', 'finalFields',
       'synchronizedFields', 'nosi', 'loc', 'returnQty', 'loopQty',
       'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty',
       'stringLiteralsQty', 'numbersQty', 'assignmentsQty',
       'mathOperationsQty', 'variablesQty', 'maxNestedBlocks',
       'anonymousClassesQty', 'subClassesQty', 'lambdasQty', 'uniqueWordsQty',
       'modifiers']

depth = 3
for project_name in project_names:
    rootdir += project_name
    #print(rootdir)
    for root, dirs, files in os.walk(rootdir):
        for file in files:
            
            dir_arr.append(rootdir + '/' + str(file))
            
    for element in dir_arr:
        ver = str(element).replace('_class.csv', '')
        ver = ver.split('-')[-1]
        
        if '_class.csv' in element:
            
            df_sum = pd.read_csv(element)
            current_element_sum = df_sum.sum()
            
            df_max = pd.read_csv(element)
            current_element_max = df_max.max()
            
            df_std = pd.read_csv(element)
            current_element_std = df_std.std()
            
            df_mean = pd.read_csv(element)
            current_element_mean = df_mean.mean()
            
            tmp_sum = []
            tmp_max = []
            tmp_std = []
            tmp_mean = []
            
            tmp_sum.append(project_name)
            tmp_sum.append(ver)
            
            tmp_max.append(project_name)
            tmp_max.append(ver)
            
            tmp_std.append(project_name)
            tmp_std.append(ver)
            
            tmp_mean.append(project_name)
            tmp_mean.append(ver)
            
            for element in ck_metrics:
                tmp_sum.append(current_element_sum[element])
                tmp_max.append(current_element_max[element])
                tmp_std.append(current_element_std[element])
                tmp_mean.append(current_element_mean[element])
        
            ck_sum_arr.append(tmp_sum)
            ck_max_arr.append(tmp_max)
            ck_std_arr.append(tmp_std)
            ck_mean_arr.append(tmp_mean)
        
            
        elif '_results.txt' in element:
            #print(project_name, element)
            spark_df = pd.DataFrame(read_dataframe_from_file(element, project_name))
            spark_df.columns = ['name', 'version', 'n_clusters', 'affinity', 'linkage', 'n_MoJo', 'mojoFM']
            #spark_df['mojoFM'] = (1 - spark_df['n_MoJo'].astype(int)/spark_count)*100
            #print(spark_df[spark_df.n_MoJo == spark_df.n_MoJo.min()].iloc[0])
            #print(spark_df.loc[spark_df.groupby('version').mojoFM.idxmax()].reset_index(drop=True))
    for element in spark_df.loc[spark_df.groupby('version').mojoFM.idxmax()].reset_index(drop=True).iterrows():
        tmp_arr2 = []
        tmp_arr2.append(element[1]['name'])
        tmp_arr2.append(element[1]['version'])
        tmp_arr2.append(element[1]['n_clusters'])
        tmp_arr2.append(element[1]['affinity'])
        tmp_arr2.append(element[1]['linkage'])
        tmp_arr2.append(element[1]['n_MoJo'])
        tmp_arr2.append(element[1]['mojoFM'])
        
        cluster_main_arr.append(tmp_arr2)
    #cluster_main_arr.append(spark_df.loc[spark_df.groupby('version').mojoFM.idxmax()].reset_index(drop=True))
            #cluster_main_arr.append(spark_df[spark_df.n_MoJo == spark_df.n_MoJo.min()].iloc[0])
    
    rootdir = 'Final_Results/'
    #dir_arr = []
            

apache-storm

apache-cassandra

airbnb-lottie-android

apache-isis

apache-jmeter

apache-log4j

apache-maven

apache-spark

apache-tomcat

rzwitserloot-lombok



### For clustering

In [250]:
final_clustering_result = pd.DataFrame(cluster_main_arr)
final_clustering_result.columns = ['name', 'version', 'n_clusters', 'affinity', 'linkage', 'n_MoJo', 'mojoFM']

In [251]:
final_clustering_result.head()

,name,version,n_clusters,affinity,linkage,n_MoJo,mojoFM
0,apache-storm,1.1.0,100,l2,single,1568,23.698297
1,apache-storm,1.1.1,50,euclidean,single,1574,23.406326
2,apache-storm,1.1.2,50,cosine,single,508,75.279805
3,apache-storm,1.1.3,50,cosine,single,511,75.133820
4,apache-storm,1.1.4,50,cosine,single,505,75.425791


In [252]:
final_clustering_result['name'].unique()


array(['apache-storm', 'apache-cassandra', 'airbnb-lottie-android',
       'apache-isis', 'apache-jmeter', 'apache-log4j', 'apache-maven',
       'apache-spark', 'apache-tomcat', 'rzwitserloot-lombok'],
      dtype=object)

In [253]:
final_clustering_result.to_csv('clustering_result_10_ver.csv', index=False)

### For CK Metrics

In [254]:
# For SUM CK Metric

ck_sum_df = pd.DataFrame(ck_sum_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_sum_df.columns = final_ck_columns
ck_sum_df.head()

ck_sum_df.to_csv('ck_sum_10_ver.csv', index=False)

In [255]:
# For Max CK Metric

ck_max_df = pd.DataFrame(ck_max_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_max_df.columns = final_ck_columns
ck_max_df.head()

ck_max_df.to_csv('ck_max_10_ver.csv', index=False)

In [256]:
# For SUM CK Metric

ck_std_arr = pd.DataFrame(ck_std_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_std_arr.columns = final_ck_columns
ck_std_arr.head()

ck_std_arr.to_csv('ck_std_10_ver.csv', index=False)

In [257]:
# For SUM CK Metric

ck_mean_arr = pd.DataFrame(ck_mean_arr)
final_ck_columns = ['project_name', 'version']
for element in ck_metrics:
    final_ck_columns.append(element)
ck_mean_arr.columns = final_ck_columns
ck_mean_arr.head()

ck_mean_arr.to_csv('ck_mean_10_ver.csv', index=False)

In [241]:
test.project_name.unique()

array(['apache-maven', 'apache-storm'], dtype=object)

In [242]:
test.head(100)

,project_name,version,cbo,wmc,dit,rfc,lcom,totalMethods,staticMethods,publicMethods,...,numbersQty,assignmentsQty,mathOperationsQty,variablesQty,maxNestedBlocks,anonymousClassesQty,subClassesQty,lambdasQty,uniqueWordsQty,modifiers
0,apache-maven,3.3.8,6094,11950,1524,12186,89882,6697,310,4480,...,2107,8922,1324,7185,677,47,90,0,45799,28843
1,apache-maven,3.3.9,6094,11951,1524,12187,89882,6697,310,4480,...,2107,8922,1324,7185,677,47,90,0,45799,28843
2,apache-maven,3.5.0,6155,12113,1537,12382,90671,6778,317,4539,...,2228,9094,1355,7344,682,51,91,0,46546,28881
3,apache-maven,3.5.1,6182,12152,1540,12431,90726,6801,315,4555,...,2249,9129,1355,7378,682,51,94,0,46826,28884
4,apache-maven,3.5.2,6179,12149,1539,12416,90734,6798,316,4553,...,2249,9117,1355,7366,682,51,94,0,46862,28900
5,apache-maven,3.5.3,6186,12196,1543,12482,91376,6813,316,4517,...,2293,9176,1362,7424,688,46,94,0,47171,28902
6,apache-maven,3.5.4,6192,12205,1545,12510,91557,6817,316,4519,...,2308,9191,1364,7439,688,46,94,0,47256,28903
7,apache-maven,3.6.0,6190,12207,1547,12561,91567,6817,316,4519,...,2320,9199,1367,7442,687,48,94,0,47320,28904
8,apache-maven,3.6.1,6207,12294,1548,12681,92991,6849,317,4537,...,2359,9311,1371,7542,688,49,96,0,47560,28905
9,apache-maven,3.6.2,6338,12316,1556,12767,93798,6897,322,4571,...,2388,9369,1308,7582,692,54,119,0,48057,28912
